In [41]:
import pandas as pd 
from sklearn.metrics import cohen_kappa_score

In [42]:
data = pd.read_csv('pilot_complete.csv')

In [43]:
data2 = pd.read_csv('pilot_gemini.csv')

In [44]:
data2

,username,user,location,gender-google
0,zhiqings,Zhiqing Sun,USA,Unknown\n
1,jysohn23,Daniel JinYoung Sohn,USA,Male\n
2,LudovicPeran,LUDOVIC PERAN,NaN,Male\n
3,jwexler,James Wexler,NaN,Male\n
4,iftenney,Ian Tenney,USA,Male\n
...,...,...,...,...
354,Mukul1,Mukul Chaudhary,NaN,Male\n
355,jorsj,Jorge Sierra,NaN,Male\n
356,awebson,Albert Webson,NaN,Unknown\n
357,lauraio,Laura Ionita,NaN,Female\n


In [45]:
gender = data[' gender'].astype(str)
data['gender-Anthropic'] = data['gender-Anthropic'].str.replace('\n', '').str.lower()
data[' Gender-google'] = data[' Gender-google'].str.replace('\n', '').str.lower()
data2['gender-google'] = data2['gender-google'].str.replace('\n', '').str.lower()
anthropic = data['gender-Anthropic'].astype(str)
google = data[' Gender-google'].astype(str)
data['gender-google-pro'] = data2['gender-google']


In [46]:
data

,username,user,ubication,gender,Gender-openai,Gender-google,gender-Anthropic,gender-google-pro
0,zhiqings,Zhiqing Sun,USA,male,Unknown,male,male,unknown
1,jysohn23,Daniel JinYoung Sohn,USA,male,Male,male,male,male
2,LudovicPeran,LUDOVIC PERAN,,male,Male,male,male,male
3,jwexler,James Wexler,,male,Male,male,male,male
4,iftenney,Ian Tenney,USA,male,Male,male,male,male
...,...,...,...,...,...,...,...,...
354,Mukul1,Mukul Chaudhary,,male,Male,male,male,male
355,jorsj,Jorge Sierra,NaN,male,Male,male,male,male
356,awebson,Albert Webson,NaN,male,Male,male,male,unknown
357,lauraio,Laura Ionita,NaN,female,Female,female,female,female


In [47]:
data[' gender'] = data[' gender'].str.strip()
data['same_gender'] = data[' gender'] == data[' Gender-google']
cohen1 = cohen_kappa_score(data[' gender'].astype(str), data['gender-Anthropic'].astype(str))
cohen2 = cohen_kappa_score(data[' gender'].astype(str), data[' Gender-google'].astype(str))
cohen3 = cohen_kappa_score(data[' gender'].astype(str), data['gender-google-pro'].astype(str))
print(cohen1.round(2), cohen2.round(2), cohen3.round(2))

0.59 0.52 0.54


In [48]:
import numpy as np
from sklearn.metrics import cohen_kappa_score
from scipy.stats import norm

def kappa_confidence_interval(y1, y2, alpha=0.05):
    # Calcular Kappa
    kappa = cohen_kappa_score(y1, y2)
    
    # Calcular la proporción de acuerdo observado (po) y esperado (pe)
    n = len(y1)
    classes = np.unique(np.concatenate([y1, y2]))
    n_classes = len(classes)
    
    confusion_matrix = np.zeros((n_classes, n_classes))
    for i, j in zip(y1, y2):
        i_index = np.where(classes == i)[0][0]
        j_index = np.where(classes == j)[0][0]
        confusion_matrix[i_index, j_index] += 1
    
    po = np.trace(confusion_matrix) / n
    pe = np.sum((confusion_matrix.sum(axis=0) * confusion_matrix.sum(axis=1)) / n**2)
    
    # Calcular la varianza
    var_kappa = (po * (1 - po)) / (n * (1 - pe)**2)
    se_kappa = np.sqrt(var_kappa)
    
    # Calcular el intervalo de confianza
    z = norm.ppf(1 - alpha / 2)
    lower = max(-1, kappa - z * se_kappa)
    upper = min(1, kappa + z * se_kappa)
    
    return kappa, lower, upper



kappa, lower, upper = kappa_confidence_interval(data[' gender'].astype(str), data['gender-Anthropic'].astype(str))
kappa1, lower1, upper1 = kappa_confidence_interval(data[' gender'].astype(str), data[' Gender-google'].astype(str))
kappa2, lower2, upper2 = kappa_confidence_interval(data[' gender'].astype(str), data['gender-google-pro'].astype(str))
print(f"Kappa: {kappa:.3f}")
print(f"Intervalo de confianza al 95%: ({lower:.3f}, {upper:.3f})")
print(f"Kappa: {kappa1:.3f}")
print(f"Intervalo de confianza al 95%: ({lower1:.3f}, {upper1:.3f})")
print(f"Kappa: {kappa2:.3f}")
print(f"Intervalo de confianza al 95%: ({lower2:.3f}, {upper2:.3f})")


Kappa: 0.592
Intervalo de confianza al 95%: (0.510, 0.674)
Kappa: 0.521
Intervalo de confianza al 95%: (0.435, 0.607)
Kappa: 0.536
Intervalo de confianza al 95%: (0.455, 0.617)
